In [122]:
import pandas as pd
from datetime import datetime
import sqlite3
import json

In [123]:
assinantesAtivos = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AssinantesAtivos.csv", sep=",") #
assinantesCancelados = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AssinantesCancelados.csv", sep =',') #
base_gostos = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/RESPOSTAS_TRATADO_FERNANDO.csv', sep=';')#
pagamentos = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/Pagamentos.csv', sep=',') 
campanha_petisco = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/PNRresponses.csv', sep=';')
campanha_AHT = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AHTresponses.csv", sep=';')
unified_costumers = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/Clientes_unificado.csv', sep=',')
coupons = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/coupons (1).csv", sep=',')

Tratando os gostos dos clientes

In [124]:
base_gostos.isnull().sum()

id                        0
nome                      1
sobrenome                 1
data_nascimento           1
genero                    1
ddd                       1
como_conheceu             3
data_inscricao            3
rede_social               0
email                     3
assinatura                3
id.1                      3
moda                      3
pessoa_artistica          3
maquiagem                 3
brinquedos_ou_jogos       3
esportes                  3
tem_pet                   3
decorar_casa              3
alimento_saudavel         3
gastronomia               3
carro                     3
mae_ou_pai                3
bebida_alcoolica          3
cuidados_pessoais         3
tecnologia                3
influencer                3
interesses                3
consumidor_id             3
renda_id                  3
restricoes_alimentares    3
tem_cupom                 3
objetivo_id               3
id.2                      3
cidade                    5
estado              

In [134]:
base_gostos = base_gostos.dropna()

In [126]:
base_gostos.columns

Index(['id', 'nome', 'sobrenome', 'data_nascimento', 'genero', 'ddd',
       'como_conheceu', 'data_inscricao', 'rede_social', 'email', 'assinatura',
       'id.1', 'moda', 'pessoa_artistica', 'maquiagem', 'brinquedos_ou_jogos',
       'esportes', 'tem_pet', 'decorar_casa', 'alimento_saudavel',
       'gastronomia', 'carro', 'mae_ou_pai', 'bebida_alcoolica',
       'cuidados_pessoais', 'tecnologia', 'influencer', 'interesses',
       'consumidor_id', 'renda_id', 'restricoes_alimentares', 'tem_cupom',
       'objetivo_id', 'id.2', 'cidade', 'estado', 'consumidor_id.1', 'bairro'],
      dtype='object')

In [127]:
del base_gostos['id']
del base_gostos['id.2']
del base_gostos['ddd']
del base_gostos['id.1']
del base_gostos['consumidor_id.1']
del base_gostos['interesses']
del base_gostos['assinatura']
del base_gostos['consumidor_id']


In [140]:
base_gostos.head()

,nome,sobrenome,data_nascimento,genero,como_conheceu,data_inscricao,rede_social,email,moda,pessoa_artistica,...,influencer,renda_id,restricoes_alimentares,tem_cupom,objetivo_id,cidade,estado,bairro,idade,faixa_etaria
1,Olivia,Carvalho,2005-04-23,Feminino,-,2024-04-29 13:25:00.000 -0300,0,oliviaandradecontato@gmail.com,True,True,...,True,R$0 - 3.000,Não tenho restrição,False,Quero dar a minha opinião sobre produtos,Caraguatatuba,São Paulo,Loteamento Jardim Aruan,19.0,Jovens
2,Aline,Marchesotti,1990-03-27,Feminino,-,2024-04-29 13:38:00.000 -0300,1,aline.marchesotti@hotmail.com,True,True,...,False,R$3.000 - 8.000,Vegetarianismo,False,Quero dar a minha opinião sobre produtos,Campinas,São Paulo,Vila Costa e Silva,34.0,Adultos
3,Amanda,Pinheiro,1985-11-11,Feminino,-,2024-04-29 13:45:00.000 -0300,0,mandinhairon@gmail.com,True,True,...,True,R$8.000 - 16.000,Não tenho restrição,False,Quero dar a minha opinião sobre produtos,Guarulhos,São Paulo,Vila Santo Antonio,39.0,Adultos
4,Fabiana,Gomes B Chaves,1975-11-08,Feminino,-,2024-04-29 14:24:00.000 -0300,1,fotografiabiagomes@gmail.com,True,True,...,True,R$3.000 - 8.000,Não tenho restrição,False,Quero descobrir novos produtos para comprar,São Paulo,São Paulo,Chácara Inglesa,49.0,Adultos
5,Flavia,Ferraresi Campos,2004-03-30,Feminino,-,2024-04-29 14:33:00.000 -0300,0,flaviaferraresi3003@gmail.com,True,False,...,True,Superior a R$24.000,Amendoim,False,Quero dar a minha opinião sobre produtos,Piracaia,São Paulo,Centro,20.0,Adultos


In [129]:
restricoes_map = {
    1: "Lactose",
    2: "Glúten",
    3: "Soja",
    4: "Ovo",
    5: "Amendoim",
    6: "Vegetarianismo",
    7: "Veganismo",
    8: "Não tenho restrição",
    9: "Outro"
}

objetivo_map = {
    1: "Quero dar a minha opinião sobre produtos",
    2: "Quero descobrir novos produtos para comprar"
}

faixa_renda_map = {
    6: "R$0 - 3.000",
    7: "R$3.000 - 8.000",
    8: "R$8.000 - 16.000",
    9: "R$16.000 - 24.000",
    10: "Superior a R$24.000"
}

# Transformação da coluna 'restricoes_alimentares' usando JSON e o mapeamento
base_gostos['restricoes_alimentares'] = base_gostos['restricoes_alimentares'].apply(
    lambda x: ", ".join([restricoes_map[id_] for id_ in json.loads(x)["restricoes"]]) 
    if pd.notnull(x) and x.startswith("{") else x
)

# Transformação das colunas 'objetivo' e 'faixa_renda' usando o .loc
base_gostos.loc[base_gostos['objetivo_id'].notnull(), 'objetivo_id'] = base_gostos['objetivo_id'].map(objetivo_map)
base_gostos.loc[base_gostos['renda_id'].notnull(), 'renda_id'] = base_gostos['renda_id'].map(faixa_renda_map)


/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/3842484211.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Quero dar a minha opinião sobre produtos'
 'Quero dar a minha opinião sobre produtos'
 'Quero dar a minha opinião sobre produtos' ...
 'Quero descobrir novos produtos para comprar'
 'Quero dar a minha opinião sobre produtos'
 'Quero dar a minha opinião sobre produtos']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  base_gostos.loc[base_gostos['objetivo_id'].notnull(), 'objetivo_id'] = base_gostos['objetivo_id'].map(objetivo_map)
/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/3842484211.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['R$16.000 - 24.000' 'R$0 - 3.000' 'R$3.000 - 8.000' ...
 'R$8.000 - 16.000' 'R$8.000 - 16.000' 

In [130]:
base_gostos['data_nascimento'] = pd.to_datetime(base_gostos['data_nascimento'], format='%d/%m/%Y', errors='coerce')

# Calcular a idade com base na data atual
today = datetime.now()
base_gostos['idade'] =  base_gostos['data_nascimento'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)) if pd.notnull(x) else None)

In [131]:
bins = [0, 19, 59, 120]  # Limites de idade para Jovens, Adultos e Idosos
labels = ["Jovens", "Adultos", "Idosos"]  # Rótulos para as faixas etárias

# Criar a coluna 'faixa_etaria' com as faixas etárias
base_gostos['faixa_etaria'] = pd.cut(base_gostos['idade'], bins=bins, labels=labels, right=True)

In [139]:
# Manter apenas as colunas necessárias para identificar assinantes
# assinantesAtivos.rename(columns={'Customer Email': 'email'}, inplace=True)
assinantesAtivos = assinantesAtivos[['email', 'Status']]
assinantesAtivos['assinantes'] = assinantesAtivos['Status'].apply(lambda x: 1 if x == 'active' else 0)
assinantesAtivos = assinantesAtivos[['email', 'assinantes']]
base_nova = pd.merge(base_gostos, assinantesAtivos, on="email", how="left")

/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/2333494724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assinantesAtivos['assinantes'] = assinantesAtivos['Status'].apply(lambda x: 1 if x == 'active' else 0)


In [143]:
base_nova['assinantes'] = base_nova['assinantes'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [147]:
base_nova.columns

Index(['nome', 'sobrenome', 'data_nascimento', 'genero', 'como_conheceu',
       'data_inscricao', 'rede_social', 'email', 'moda', 'pessoa_artistica',
       'maquiagem', 'brinquedos_ou_jogos', 'esportes', 'tem_pet',
       'decorar_casa', 'alimento_saudavel', 'gastronomia', 'carro',
       'mae_ou_pai', 'bebida_alcoolica', 'cuidados_pessoais', 'tecnologia',
       'influencer', 'renda_id', 'restricoes_alimentares', 'tem_cupom',
       'objetivo_id', 'cidade', 'estado', 'bairro', 'idade', 'faixa_etaria',
       'assinantes'],
      dtype='object')

Tratamento dos assinantes

In [149]:
assinantesAtivos = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AssinantesAtivos.csv", sep=",")
assinantesAtivos.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Tax Percent', 'Canceled At (UTC)'],
      dtype='object')

In [150]:
assinantesAtivos.head()

,Customer ID,Customer Email,Plan,Quantity,Interval,Amount,Status,Created (UTC),Start Date (UTC),Current Period Start (UTC),Current Period End (UTC),Coupon,Tax Percent,Canceled At (UTC)
0,cus_RC1UzB1huhrZ4X,lionbranco@gmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 15:52,2024-11-10 15:52,2024-11-10 15:52,2024-12-10 15:52,NaN,NaN,NaN
1,cus_RC0ijbOaql9B2c,andressa21a@live.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 15:05,2024-11-10 15:05,2024-11-10 15:05,2024-12-10 15:05,NaN,NaN,NaN
2,cus_RBzVjVhlJtx5WQ,natholiveirar03@gmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 13:49,2024-11-10 13:49,2024-11-10 13:49,2024-12-10 13:49,NaN,NaN,NaN
3,cus_RBwHefgWZtgleQ,sabrinascandolararp@gmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 10:30,2024-11-10 10:30,2024-11-10 10:30,2024-12-10 10:30,NaN,NaN,NaN
4,cus_RBqYJB57nElUiu,joyce_p_almeida@hotmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 04:34,2024-11-10 04:34,2024-11-10 04:34,2024-12-10 04:34,NaN,NaN,NaN


In [ ]:
del assinantesAtivos['Tax Percent']

In [152]:
assinantesAtivos['Coupon'] = assinantesAtivos['Coupon'].fillna('-')
assinantesAtivos['Canceled At (UTC)']=assinantesAtivos['Canceled At (UTC)'].fillna('N/a')


In [153]:
assinantesAtivos.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Tax Percent', 'Canceled At (UTC)'],
      dtype='object')

In [155]:
assinantesCancelados.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Tax Percent', 'Canceled At (UTC)', 'Ended At (UTC)', 'Customer Name'],
      dtype='object')

In [159]:
del assinantesCancelados['Customer Name']
del assinantesCancelados['Tax Percent']

In [160]:
assinantesTotal = pd.concat([assinantesAtivos, assinantesCancelados], ignore_index=True)

Tratamento da base de 

In [ ]:
# import sqlite3
# import pandas as pd

# # Conectar ao banco de dados SQLite
# db_path = 'Experimentai.db'
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Modelo Estrela para `base_nova`
# # Tabela DimDemograficoBaseNova
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimDemograficoBaseNova (
#         id_demografico INTEGER PRIMARY KEY,
#         id INTEGER,
#         data_nascimento TEXT,
#         genero TEXT,
#         email TEXT,
#         cidade TEXT,
#         estado TEXT,
#         idade INTEGER,
#         faixa_etaria TEXT,
#         bairro TEXT
#     );
# ''')

# # Tabela DimGostoBaseNova
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimGostoBaseNova (
#         id_gosto INTEGER PRIMARY KEY,
#         bebida BOOLEAN,
#         gastronomia BOOLEAN,
#         moda BOOLEAN,
#         maquiagem BOOLEAN,
#         esporte BOOLEAN,
#         tipo_pet BOOLEAN,
#         pessoa_artistica BOOLEAN,
#         brinquedos_ou_jogos BOOLEAN,
#         tem_pet BOOLEAN,
#         decorar_casa BOOLEAN,
#         alimento_saudavel BOOLEAN,
#         carro BOOLEAN,
#         bebida_alcoolica BOOLEAN,
#         cuidados_pessoais BOOLEAN,
#         tecnologia BOOLEAN,
#         mae_ou_pai BOOLEAN,
#         influencer BOOLEAN,
#         restricoes_alimentares BOOLEAN,
#         objetivo_id INTEGER
#     );
# ''')

# # Tabela de Fato para base_nova (clienteFato)
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS clienteFato (
#         id_fato INTEGER PRIMARY KEY,
#         id_cliente INTEGER,
#         data_inscricao TEXT,
#         rede_social TEXT,
#         renda_id INTEGER,
#         objetivo_id INTEGER,
#         faixa_etaria TEXT,
#         cidade TEXT,
#         estado TEXT,
#         bairro TEXT,
#         id_demografico INTEGER,
#         id_gosto INTEGER,
#         FOREIGN KEY(id_demografico) REFERENCES DimDemograficoBaseNova(id_demografico),
#         FOREIGN KEY(id_gosto) REFERENCES DimGostoBaseNova(id_gosto)
#     );
# ''')

# # Modelo Estrela para AHT
# # Tabela DimDemograficoAHT
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimDemograficoAHT (
#         id_demografico INTEGER PRIMARY KEY,
#         nome TEXT,
#         email TEXT,
#         idade INTEGER,
#         genero TEXT,
#         renda TEXT
#     );
# ''')

# # Tabela DimGostoAHT
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimGostoAHT (
#         id_gosto INTEGER PRIMARY KEY,
#         bebida BOOLEAN,
#         gastronomia BOOLEAN,
#         moda BOOLEAN,
#         maquiagem BOOLEAN,
#         esporte BOOLEAN
#     );
# ''')

# # Tabela de Fato para AHT (FatoAHT)
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS FatoAHT (
#         id_fato_aht INTEGER PRIMARY KEY,
#         id_cliente INTEGER,
#         ja_conhecia BOOLEAN,
#         nota INTEGER,
#         intencao_compra BOOLEAN,
#         onde_gcomprar TEXT,
#         quanto_pagaria REAL,
#         trocar_bebida_por_aht BOOLEAN,
#         recomendar_aht BOOLEAN,
#         id_demografico INTEGER,
#         id_gosto INTEGER,
#         FOREIGN KEY(id_demografico) REFERENCES DimDemograficoAHT(id_demografico),
#         FOREIGN KEY(id_gosto) REFERENCES DimGostoAHT(id_gosto)
#     );
# ''')

# # Modelo Estrela para Petisco
# # Tabela DimDemograficoPetisco
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimDemograficoPetisco (
#         id_demografico INTEGER PRIMARY KEY,
#         email TEXT
#     );
# ''')

# # Tabela DimGostoPetisco
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimGostoPetisco (
#         id_gosto INTEGER PRIMARY KEY,
#         tipo_pet BOOLEAN
#     );
# ''')

# # Tabela de Fato para Petisco (FatoPetiscos)
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS FatoPetiscos (
#         id_fato_petiscos INTEGER PRIMARY KEY,
#         id_cliente INTEGER,
#         conhecia_produto BOOLEAN,
#         reacao_pet TEXT,
#         tamanho_petisco TEXT,
#         motivo_dar_petisco TEXT,
#         fonte_info_produtos TEXT,
#         atracao_promocao TEXT,
#         avaliacao_custo_beneficio TEXT,
#         local_compra_petiscos TEXT,
#         fator_importante_compra_online TEXT,
#         gasto_mensal_petiscos REAL,
#         disposto_pagar_mais BOOLEAN,
#         motivo_mudanca_fornecedor TEXT,
#         intencao_compra_produto BOOLEAN,
#         frequencia_dar_petisco TEXT,
#         fatores_importantes_escolha TEXT,
#         intencao_proxima_compra TEXT,
#         id_demografico INTEGER,
#         id_gosto INTEGER,
#         FOREIGN KEY(id_demografico) REFERENCES DimDemograficoPetisco(id_demografico),
#         FOREIGN KEY(id_gosto) REFERENCES DimGostoPetisco(id_gosto)
#     );
# ''')

# # Commit para salvar a criação das tabelas
# conn.commit()

# # 2. Inserir os Dados dos Sub-DataFrames em Cada Tabela

# # Função para inserir DataFrames nas tabelas correspondentes
# def insert_data(df, table_name):
#     df.to_sql(table_name, conn, if_exists='append', index=False)

# # Modelo Estrela para base_nova
# insert_data(dim_demografico_base_nova, 'DimDemograficoBaseNova')
# insert_data(dim_gosto_base_nova, 'DimGostoBaseNova')
# insert_data(base_nova[['id', 'data_inscricao', 'rede_social', 'renda_id', 'objetivo_id', 
#                       'faixa_etaria', 'cidade', 'estado', 'bairro', 'id_demografico', 'id_gosto']].rename(columns={'id': 'id_cliente'}), 
#             'clienteFato')

# # Modelo Estrela para AHT
# insert_data(dim_demografico_aht, 'DimDemograficoAHT')
# insert_data(dim_gosto_aht, 'DimGostoAHT')
# insert_data(feedback_campanhas_aht[['nome', 'ja_conhecia', 'nota', 'intencao_compra', 'onde_gcomprar', 
#                                     'quanto_pagaria', 'trocar_bebida_por_aht', 'recomendar_aht', 
#                                     'id_demografico', 'id_gosto']].rename(columns={'nome': 'id_cliente'}), 
#             'FatoAHT')

# # Modelo Estrela para Petisco
# insert_data(dim_demografico_petisco, 'DimDemograficoPetisco')
# insert_data(dim_gosto_petisco, 'DimGostoPetisco')
# insert_data(feedback_campanhas_petisco[['email', 'conhecia_produto', 'reacao_pet', 'tamanho_petisco', 
#                                         'motivo_dar_petisco', 'fonte_info_produtos', 'atracao_promocao', 
#                                         'avaliacao_custo_beneficio', 'local_compra_petiscos', 
#                                         'fator_importante_compra_online', 'gasto_mensal_petiscos', 
#                                         'disposto_pagar_mais', 'motivo_mudanca_fornecedor', 
#                                         'intencao_compra_produto', 'frequencia_dar_petisco', 
#                                         'fatores_importantes_escolha', 'intencao_proxima_compra', 
#                                         'id_demografico', 'id_gosto']].rename(columns={'email': 'id_cliente'}), 
#             'FatoPetiscos')

# # Fechar a conexão com o banco de dados
# conn.close()
